In [1]:
import os
import pymysql

In [2]:
cafe24_db = pymysql.connect(
    user='csp', 
    passwd=' ', 
    host='5 ', 
    db='statics', 
    charset='utf8'
)
cursor = cafe24_db.cursor(pymysql.cursors.DictCursor)

In [5]:
down_path = 'download_20201215_20201023'
merge_path = './merge_{}'.format(down_path)

if os.path.exists(merge_path) == False:
    os.mkdir(merge_path)

# 일 단위 판매된 품목

In [5]:
a = pd.read_csv('./{}/day_product_option.csv'.format(down_path))
a['옵션'] = a['옵션'].fillna(a['상품명'])

In [6]:
a.tail()

,상품명,옵션,판매수량,판매가,판매합계,date
1499,문라이트,문라이트,1,9900,9900,2020-10-24
1500,언더더씨,언더더씨,1,14900,14900,2020-10-24
1501,프렌즈포에버,프렌즈포에버,1,14900,14900,2020-10-24
1502,킹덤,킹덤,1,14900,14900,2020-10-23
1503,더크로스,더크로스,1,9900,9900,2020-10-23


In [7]:
tmp = a
sql = "INSERT INTO `day_product_option`({}) VALUES ({});".format(','.join(["`{}`".format(col) for col in tmp.columns]), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 방문자 데이터

In [8]:
def dropColumn(df, columns):
    for dcol in columns:
        if dcol in df.columns:
            df = df.drop(columns=[dcol])
    return df

In [9]:
a = pd.read_csv('./{}/rpt_con_pvi_tab.csv'.format(down_path))
a = dropColumn(a,[ '전날평균','증감', '비교값'])
        
b = pd.read_csv('./{}/rpt_con_vis_tab.csv'.format(down_path))
b = dropColumn(b,[ '전날평균','증감', '재방문', '비교값'])

c = pd.read_csv('./{}/rpt_con_uni_tab.csv'.format(down_path))
c = dropColumn(c,[ '전날평균','증감', '전체방문', '비교값'])

# a = pd.read_csv('./{}/rpt_con_pvi_tab.csv'.format(down_path)).drop(columns=[ '전날평균','증감'])
# b = pd.read_csv('./{}/rpt_con_vis_tab.csv'.format(down_path)).drop(columns=[ '전날평균','증감','재방문'])
# c = pd.read_csv('./{}/rpt_con_uni_tab.csv'.format(down_path)).drop(columns=[ '전날평균','증감','전체방문'])

a['페이지뷰'] = a['페이지뷰'].map(lambda x: x.replace(',','')).astype(int)
tmp = pd.merge(a, b, on=['일시'], how='left')
tmp = pd.merge(tmp, c, on=['일시'], how='left')
#tmp['date'] = pd.to_datetime(tmp['date'])
tmp = tmp.sort_values('일시', ascending=[False])
tmp  = tmp.rename(columns={'일시':'date'})

tmp['전체방문수'] = tmp['전체방문수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
tmp['순방문수'] = tmp['순방문수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
tmp['재방문수'] = tmp['재방문수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
tmp['처음방문'] = tmp['처음방문'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
tmp.to_csv('./{}/hour_view_visit.csv'.format( merge_path), index=False, encoding='utf-8')

In [12]:
tmp.tail()

,date,페이지뷰,전체방문수,처음방문,순방문수,재방문수
4,2020-10-27,1534,432,143,343,289
3,2020-10-26,1470,509,180,435,329
2,2020-10-25,1131,407,151,358,256
1,2020-10-24,907,329,133,294,196
0,2020-10-23,931,333,116,280,217


In [13]:
sql = "INSERT INTO `hour_view_visit`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 결제수단별 금액

In [14]:
d = pd.read_csv('./{}/rpt_sal_pmd_tab.csv'.format(down_path)).drop(columns=[ '합계'])
d = d.rename(columns={'계좌이체\n가상계좌': '계좌이체가상계좌'})
d['현금'] = d['현금'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
d['신용카드'] = d['신용카드'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
d['예치금'] = d['예치금'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
d['적립금'] = d['적립금'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
d['계좌이체가상계좌'] = d['계좌이체가상계좌'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
d['간편결제'] = d['간편결제'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
d['휴대폰'] = d['휴대폰'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
d['기타'] = d['기타'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

tmp = d
tmp['hour'] = tmp['일시'].str.replace('시','').astype(int)
tmp = tmp.drop(columns=['일시'])
#tmp = tmp.sort_values(['date', 'hour'], ascending=[True, True])
tmp.to_csv('./{}/hour_payment.csv'.format(merge_path), index=False, encoding='utf-8')

In [16]:
sql = "INSERT INTO `hour_payment`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 구매자(재방문, 처음구매) 매출

In [17]:
e = pd.read_csv('./{}/rpt_buy_vis_tab.csv'.format(down_path)).drop(columns=[ '구매율','방문자수'])
f = pd.read_csv('./{}/rpt_buy_res_tab.csv'.format(down_path)).drop(columns=[ '1', '4','7','10'])
f.columns= ['일시', '처음구매상품수','재구매상품수','처음구매건수','재구매건수','처음구매액','재구매액','date']
g = pd.read_csv('./{}/rpt_buy_rec_tab.csv'.format(down_path)).drop(columns=[])
g.columns=['일시', '처음방문수', '재방문수', '처음방문구매건수', '재방문구매건수', '처음방문구매액', '재방문구매액', '처음방문구매율', '재방문구매율', 'date']
g = g.drop(columns=['처음방문수','재방문수', '재방문구매율', '처음방문구매율'])


f['처음구매액'] = f['처음구매액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
f['재구매액'] = f['재구매액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

g['처음방문구매액'] = g['처음방문구매액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
g['재방문구매액'] = g['재방문구매액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

tmp = pd.merge(e, f, on=['일시','date'], how='left')
tmp = pd.merge(tmp, g, on=['일시','date'], how='left')

tmp['hour'] = tmp['일시'].str.replace('시','').astype(int)
#tmp['date'] = pd.to_datetime(tmp['date'])
tmp = tmp.drop(columns=['일시'])
#tmp = tmp.sort_values(['date', 'hour'], ascending=[True, True])
tmp.to_csv('./{}/hour_first_re_count_amout.csv'.format(merge_path), index=False, encoding='utf-8')

In [19]:
sql = "INSERT INTO `hour_first_re_count_amout`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 품목별 재구매, 처음구매 매출

In [20]:
for path, dirs, files in os.walk('./{}'.format(down_path)):
    print(files)
    break
    
files = ['./{}/{}'.format(down_path,file) for file in files if file.startswith('rpt_buy_res_tab-')]

['rpt_prd_cas_tab-20201126.csv', 'rpt_buy_res_tab-20201202.csv', 'rpt_buy_res_tab-20201028.csv', 'rpt_pth_egd_tab-20201024.csv', 'rpt_pth_egd_tab-20201030.csv', 'rpt_pth_egd_tab-20201031.csv', 'rpt_pth_egd_tab-20201025.csv', 'rpt_buy_res_tab-20201029.csv', 'rpt_buy_res_tab-20201203.csv', 'rpt_prd_cas_tab-20201127.csv', 'rpt_prd_cas_tab.csv', 'rpt_prd_cas_tab-20201125.csv', 'rpt_prd_cas_tab-20201119.csv', 'rpt_buy_res_tab-20201201.csv', 'rpt_buy_res_tab-20201215.csv', 'rpt_vis_nmb_tab.csv', 'rpt_pth_egd_tab-20201027.csv', 'rpt_pth_egd_tab-20201026.csv', 'rpt_rou_dth_tab.csv', 'rpt_buy_res_tab-20201214.csv', 'rpt_prd_cas_tab-20201118.csv', 'rpt_prd_cas_tab-20201124.csv', 'rpt_prd_cas_tab-20201130.csv', 'rpt_prd_cas_tab-20201108.csv', 'rpt_prd_cas_tab-20201120.csv', 'rpt_buy_res_tab-20201204.csv', 'rpt_buy_res_tab-20201210.csv', 'rpt_pth_egd_tab-20201208.csv', 'rpt_pth_egd_tab-20201023.csv', 'rpt_pth_egd_tab-20201209.csv', 'rpt_buy_res_tab-20201211.csv', 'rpt_buy_res_tab-20201205.csv', 'r

In [21]:
tmp_li = []
for file in files: 
    tmp= pd.read_csv(file).drop(columns=['재구매율','재구매율.1'])
    tmp.columns = ['상품명','처음구매건수','재구매건수','처음구매액','재구매액','date','hour']
    try:
        tmp['처음구매액'] = tmp['처음구매액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
    except Exception as e:
        print(e)
        pass
    try:
        tmp['재구매액'] = tmp['재구매액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
    except Exception as e:
        print(e)
        pass

    tmp_li.append(tmp)
    
tmp = pd.concat(tmp_li).reset_index(drop=True)
tmp['hour'] = tmp['hour'].str.replace('시','').astype(int)
tmp = tmp.sort_values(['date', 'hour'], ascending=[True, True])
tmp.to_csv('./{}/hour_product.csv'.format(merge_path),index=False)

In [23]:
sql = "INSERT INTO `hour_product`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 품목별 노출수, 장바구니 담김수, 포기수 매출액

In [24]:
a = pd.read_csv('./{}/rpt_bsk_pin_tab.csv'.format(down_path))
a = a.drop(columns=['담긴수:노출수', '담김율'])
if a['노출수'].dtype.name != 'int64':
    a['노출수'] = a['노출수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
if a['담긴수'].dtype.name != 'int64':
    a['담긴수'] = a['담긴수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

b = pd.read_csv('./{}/rpt_bsk_out_tab.csv'.format(down_path))
b = b.drop(columns=['담긴수','포기율','담긴수:포기수'])
if b['포기수'].dtype.name != 'int64':
    b['포기수'] = b['포기수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
     
        
c = pd.read_csv('./{}/rpt_prd_sal_tab.csv'.format(down_path))
c = c.drop(columns=['담긴수','노출수', '판매율','담김/노출','판매/노출'])
c['상품명'] = c['상품명'].map(lambda x: x.replace('\n',''))
c['판매금액'] =c['판매금액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

tmp = pd.merge(a, c, on=['상품명','date'], how='outer')
tmp = tmp.merge(b,on=['상품명','date'], how='outer')
tmp = tmp.fillna(0)
tmp = tmp.sort_values(['date'], ascending=[False])
tmp.to_csv('./{}/day_product.csv'.format(merge_path),index=False)

In [26]:
sql = "INSERT INTO `day_product`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 카테고리별 판매금액과 페이지뷰 ()

In [13]:
a = pd.read_csv('./{}/rpt_prd_cas_tab.csv'.format(down_path))
a['상품군명(카테고리번호)'] = a['상품군명(카테고리번호)'].map(lambda x: x.replace('\n',''))
a['카테고리번호'] = a['상품군명(카테고리번호)'].map(lambda x: x.split(' (')[-1])
a['카테고리번호'] = a['카테고리번호'].map(lambda x: x.replace('cate_no=', '').replace(')',''))
a['카테고리명'] = a['상품군명(카테고리번호)'].map(lambda x: x.split(' (')[0])

b = pd.read_csv('./{}/rpt_pag_fad_tab.csv'.format(down_path))
b['카테고리명'] = b['카테고리명'].map(lambda x: x.replace('\n',''))
b['페이지뷰'] = b['페이지뷰'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

In [15]:
cate_no_li = [118, 119, 122, 123, 139, 140, 144, 145]

tmp = a.merge(b, on=['카테고리명','date'], how='left')
tmp = tmp[tmp['카테고리번호']!='']

tmp = tmp.drop(columns=['상품리스트','판매율','비율','처음접속당PV','노출수', '상품군명(카테고리번호)']).fillna(0)
tmp['카테고리번호'] = tmp['카테고리번호'].astype(int)
tmp = tmp[tmp['카테고리번호'].isin(cate_no_li)]
tmp['판매금액'] = tmp['판매금액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
tmp['처음접속수'] = tmp['처음접속수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
tmp['담긴수'] = tmp['담긴수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

tmp = tmp.reset_index(drop=True)
tmp.to_csv('./{}/day_category.csv'.format(merge_path),index=False)

In [17]:
sql = "INSERT INTO `day_category`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 상품 카테고리별 판매건수 판매금액

In [32]:
for path, dirs, files in os.walk('./{}'.format(down_path)):
    print(files)
    break
files = ['./{}/{}'.format(down_path, file) for file in files if file.startswith('rpt_prd_cas_tab-')]

['rpt_prd_cas_tab-20201126.csv', 'rpt_buy_res_tab-20201202.csv', 'rpt_buy_res_tab-20201028.csv', 'rpt_pth_egd_tab-20201024.csv', 'rpt_pth_egd_tab-20201030.csv', 'rpt_pth_egd_tab-20201031.csv', 'rpt_pth_egd_tab-20201025.csv', 'rpt_buy_res_tab-20201029.csv', 'rpt_buy_res_tab-20201203.csv', 'rpt_prd_cas_tab-20201127.csv', 'rpt_prd_cas_tab.csv', 'rpt_prd_cas_tab-20201125.csv', 'rpt_prd_cas_tab-20201119.csv', 'rpt_buy_res_tab-20201201.csv', 'rpt_buy_res_tab-20201215.csv', 'rpt_vis_nmb_tab.csv', 'rpt_pth_egd_tab-20201027.csv', 'rpt_pth_egd_tab-20201026.csv', 'rpt_rou_dth_tab.csv', 'rpt_buy_res_tab-20201214.csv', 'rpt_prd_cas_tab-20201118.csv', 'rpt_prd_cas_tab-20201124.csv', 'rpt_prd_cas_tab-20201130.csv', 'rpt_prd_cas_tab-20201108.csv', 'rpt_prd_cas_tab-20201120.csv', 'rpt_buy_res_tab-20201204.csv', 'rpt_buy_res_tab-20201210.csv', 'rpt_pth_egd_tab-20201208.csv', 'rpt_pth_egd_tab-20201023.csv', 'rpt_pth_egd_tab-20201209.csv', 'rpt_buy_res_tab-20201211.csv', 'rpt_buy_res_tab-20201205.csv', 'r

In [33]:
tmp_li = []
for file in files: 
    tmp= pd.read_csv(file).drop(columns=['판매율'])
    tmp = tmp.rename(columns={'hour':'category'})
    
    tmp['판매금액'] = tmp['판매금액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
    
    tmp = tmp[(tmp['category'] != '-') & tmp['category'].notna()]
    tmp_li.append(tmp)
    
tmp = pd.concat(tmp_li).reset_index(drop=True)
tmp['노출수'] = tmp['노출수'].map(lambda x: int(x.replace(',', '')) if type(x) == str else x )
tmp['담긴수'] = tmp['담긴수'].map(lambda x: int(x.replace(',', '')) if type(x) == str else x )
tmp = tmp.sort_values(['date'])
tmp.to_csv('./{}/day_category_product.csv'.format(merge_path),index=False)

In [35]:
sql = "INSERT INTO `day_category_product`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 접속 도메인 별 방문수와 매출액

In [37]:
bigsite = ['facebook','instagram','naver','daum','google', 'youtube','brunch','bing','yahoo','nate','telegram',
           'mashupangels','zum','marieclairekorea','sparklabs','t.co','bookmark','baidu','instanttattoo.store','instant.tattoo',
          'girlstyle', 'instanttattoo.myshopify.com', 'slack', 'mailchimp', 'lofficiel', 'wconcept', 'amorepacific']
 
def checkdomain(x):
    res = None
    
    for site in bigsite:
        if site in x.lower():
            res = site
            break
            
    return res

In [38]:
a = pd.read_csv('./{}/rpt_pth_ufl_tab.csv'.format(down_path))
b = pd.read_csv('./{}/rpt_pth_dod_tab.csv'.format(down_path))
a['상세URL'] = a['상세URL'].map(lambda x : x.replace('\n',''))

b['domain'] = b['방문도메인'].map(checkdomain)
a['domain'] = a['상세URL'].map(checkdomain)

a = a[a['domain'].notna()]
b = b[b['domain'].notna()]

#a = a.drop(columns=['방문비율'])
a['방문수'] = a['방문수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

a = dropColumn(a, ['방문비율'])
b = dropColumn(b, ['방문비율','구매율','방문당매출','구매당매출'] )
b['방문수'] = b['방문수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
b['매출액'] = b['매출액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

a.to_csv('./{}/day_url.csv'.format(merge_path), index=False)
b.to_csv('./{}/day_domain.csv'.format(merge_path),index=False)

In [40]:
tmp = a
sql = "INSERT INTO `day_url`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

tmp = b
sql = "INSERT INTO `day_domain`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 접속 도메인별 매출건수, 매출액, 방문건수

In [43]:
a = pd.read_csv('./{}/rpt_pth_egd_tab.csv'.format(down_path))
a= a.drop(columns=['키워드','구매율','방문당매출','구매당매출'])
a['매출액'] = a['매출액'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
#a.to_csv('./merge_data/data_day_searchengine.csv',index=False)

In [44]:
b_li = []
for v in ['방문수','구매건수','매출액']:
    b = a.pivot_table(index="date", columns='검색엔진',values=v, aggfunc='sum', fill_value=0)
    b = b.reset_index()
    b.columns.name = None

    ncols = {}
    for col in b.columns:
        if col == 'date':
            continue 
        ncol = '{}_{}'.format(col, v)
        ncols[col] = ncol

    b = b.rename(columns=ncols)
    b_li.append(b)
    
b = pd.concat(b_li, axis=1)
b = b.loc[:, ~b.columns.duplicated()]
b = b[sorted(b.columns)]

b.to_csv('./{}/day_search_engine.csv'.format(merge_path),index=False)

In [47]:
usingc = set(b.columns).intersection(
set(['date', 
       '구글(미국)_구매건수', '구글(미국)_매출액', '구글(미국)_방문수', '네이버_구매건수', '네이버_매출액', 
       '네이버_방문수', '다음_구매건수', '다음_매출액', '다음_방문수']) )

tmp = b[usingc]

for col in [
       '구글(미국)_구매건수', '구글(미국)_매출액', '구글(미국)_방문수', '네이버_구매건수', '네이버_매출액',
       '네이버_방문수', '다음_구매건수', '다음_매출액', '다음_방문수']:
     
    if col in tmp.columns :
        tmp[col] =    tmp[col].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)

In [48]:
sql = "INSERT INTO `day_search_engine`({}) VALUES ({});".format(','.join(["`{}`".format(col) for col in tmp.columns]), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 검색어별 매출

In [49]:
for path, dirs, files in os.walk('./{}'.format(down_path)):
    #print(files)
    break
files = ['./{}/{}'.format(down_path, file) for file in files if file.startswith('rpt_pth_egd_tab-')]

tmp_li = []
for file in files: 
    tmp= pd.read_csv(file).drop(columns=['구매율','방문당매출','구매당매출'])
    
    
    tmp['매출액'] = tmp['매출액'].astype(str).map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
    tmp_li.append(tmp)
    
tmp = pd.concat(tmp_li).reset_index(drop=True)
tmp = tmp.sort_values('date')
tmp['방문수'] = tmp['방문수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
tmp.to_csv('./{}/day_search_keyword.csv'.format(merge_path),index=False)

In [51]:
sql = "INSERT INTO `day_search_keyword`({}) VALUES ({});".format(','.join(tmp.columns), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 재방문 기간

In [52]:
a = pd.read_csv('./{}/rpt_vis_rec_tab.csv'.format(down_path)).drop(columns=['비율(%)'])
columns = a['재방문간격'].unique()
a = a.pivot_table(index=['date'], columns=['재방문간격'], values='방문수', aggfunc='sum')
a = a[columns]
a = a.reset_index()
for col in columns:
    try:
        a[col] = a[col].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
    except:
        pass
    
a.columns.name = None
a.to_csv('./{}/day_revisit_period.csv'.format(merge_path), index=False)

In [53]:
tmp = a
sql = "INSERT INTO `day_revisit_period`({}) VALUES ({});".format(','.join(["`{}`".format(col) for col in tmp.columns]), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 재방문 횟수

In [54]:
a = pd.read_csv('./{}/rpt_vis_con_tab.csv'.format(down_path)).drop(columns=['비율(%)'])
columns = a['방문횟수'].unique()
a = a.pivot_table(index=['date'], columns=['방문횟수'], values='IP수', aggfunc='sum')
a = a[columns]
a = a.reset_index()

for col in columns:
    try:
        a[col] = a[col].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
    except:
        pass
 
    
a.columns.name = None
#a['date'] = pd.to_datetime(a['date'])
a.to_csv('./{}/day_visit_count.csv'.format(merge_path), index=False)

In [56]:
tmp = a
sql = "INSERT INTO `day_visit_count`({}) VALUES ({});".format(','.join(["`{}`".format(col) for col in tmp.columns]), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 이동경로길이

In [57]:
a = pd.read_csv('./{}/rpt_rou_dth_tab.csv'.format(down_path)).drop(columns=['비율'])
columns = a['이동경로깊이'].unique()
a = a.pivot_table(index=['date'], columns=['이동경로깊이'], values='방문수', aggfunc='sum')
a = a[columns]
a = a.reset_index()

for col in columns:
    try:
        a[col] = a[col].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
    except:
        pass
 
    
a.columns.name = None
#a['date'] = pd.to_datetime(a['date'])
a.to_csv('./{}/day_page_depth.csv'.format(merge_path), index=False)

In [58]:
tmp = a
sql = "INSERT INTO `day_page_depth`({}) VALUES ({});".format(','.join(["`{}`".format(col) for col in tmp.columns]), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 구매단계

In [ ]:
a = pd.read_csv('./{}/rpt_sal_stp_tab.csv'.format(down_path)).drop(columns=['전환율(%)', '이탈횟수','이탈율(%)'])
columns = a['구매단계'].unique()
a = a.pivot_table(index=['date'], columns=['구매단계'], values='접속횟수(회)', aggfunc='sum')
a = a[columns]
a = a.reset_index()
for col in columns:
    try:
        a[col] = a[col].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
    except:
        pass
 
    
a.columns.name = None
#a['date'] = pd.to_datetime(a['date'])
a.to_csv('./{}/day_pay_step.csv'.format(merge_path), index=False)

In [60]:
tmp = a
sql = "INSERT INTO `day_pay_step`({}) VALUES ({});".format(','.join(["`{}`".format(col) for col in tmp.columns]), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

# 신규 가입자

In [61]:
a = pd.read_csv('./{}/rpt_vis_nmb_tab.csv'.format(down_path)).drop(columns=['비율','일자'])
#a['date'] = pd.to_datetime(a['date'])
a['회원수'] = a['회원수'].map(lambda x: x.replace(',','') if type(x)==str else x).astype(int)
a.to_csv('./{}/day_new_member.csv'.format(merge_path), index=False)

In [62]:
tmp = a
sql = "INSERT INTO `day_new_member`({}) VALUES ({});".format(','.join(["`{}`".format(col) for col in tmp.columns]), 
                                                             ','.join(['%s']*len(tmp.columns)))

data = [list(v) for v in tmp.values]
cursor.executemany(sql, data)
cafe24_db.commit()

In [63]:
cursor.close()
cafe24_db.close()